# ColBERTv2: Indexing & Search Notebook

We start by importing the relevant classes. As we'll see below, `Indexer` and `Searcher` are the key actors here. 

In [7]:
import os
import sys
sys.path.insert(0, '../')

from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

The workflow here assumes an IR dataset: a set of queries and a corresponding collection of passages.

The classes `Queries` and `Collection` provide a convenient interface for working with such datasets.

We will use the *dev set* of the **LoTTE benchmark** we recently introduced in the ColBERTv2 paper. The dev and test sets contain several domain-specific corpora, and we'll use the smallest dev set corpus, namely `lifestyle:dev`.

In [2]:
!mkdir -p downloads/

# ColBERTv2 checkpoint trained on MS MARCO Passage Ranking (388MB compressed)
!wget https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz -P downloads/
!tar -xvzf downloads/colbertv2.0.tar.gz -C downloads/

# The LoTTE dev and test sets (3.4GB compressed)
!wget https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/lotte.tar.gz -P downloads/
!tar -xvzf downloads/lotte.tar.gz -C downloads/

--2023-02-12 15:54:09--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz
正在解析主机 downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
正在连接 downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 405924985 (387M) [application/octet-stream]
正在保存至: ‘downloads/colbertv2.0.tar.gz’

colbertv2.0.tar.gz  100%[===================>] 387.12M  5.02MB/s    用时 76s     

2023-02-12 15:55:26 (5.12 MB/s) - 已保存 ‘downloads/colbertv2.0.tar.gz’ [405924985/405924985])

colbertv2.0/
colbertv2.0/artifact.metadata
colbertv2.0/vocab.txt
colbertv2.0/tokenizer.json
colbertv2.0/special_tokens_map.json
colbertv2.0/tokenizer_config.json
colbertv2.0/config.json
colbertv2.0/pytorch_model.bin
--2023-02-12 15:55:28--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/lotte.tar.gz
正在解析主机 downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
正在连接 downloads.cs.stanford.edu (downloads.cs.stanfo

In [31]:
dataroot = 'docs/downloads/lotte'
dataset = 'lifestyle'
datasplit = 'dev'

queries = os.path.join(dataroot, dataset, datasplit, 'questions.search.tsv')
collection = os.path.join(dataroot, dataset, datasplit, 'collection.tsv')

queries = Queries(path=queries)
collection = Collection(path=collection)

f'Loaded {len(queries)} queries and {len(collection):,} passages'

print("11")
print(collection[1145])

[Feb 23, 04:42:29] #> Loading the queries from docs/downloads/lotte/lifestyle/dev/questions.search.tsv ...


FileNotFoundError: [Errno 2] No such file or directory: 'docs/downloads/lotte/lifestyle/dev/questions.search.tsv'

This loaded 417 queries and 269k passages. Let's inspect one query and one passage.

In [10]:
print(queries[24])
print()
print(collection[1])
print()

are blossom end rot tomatoes edible?

"Good Harbor. Anita Diamant's international bestseller ""The Red Tent"" brilliantly re-created the ancient world of womanhood. Diamant brings her remarkable storytelling skills to ""Good Harbor"" -- offering insight to the precarious balance of marriage and career, motherhood and friendship in the world of modern women. The seaside town of Gloucester, Massachusetts is a place where the smell of the ocean lingers in the air and the rocky coast glistens in the Atlantic sunshine. When longtime Gloucester-resident Kathleen Levine is diagnosed with breast cancer, her life is thrown into turmoil. Frightened and burdened by secrets, she meets Joyce Tabachnik -- a freelance writer with literary aspirations -- and a once-in-a-lifetime friendship is born. Joyce has just bought a small house in Gloucester, where she hopes to write as well as vacation with her family. Like Kathleen, Joyce is at a fragile place in her life. A mutual love for books, humor, and t

## Indexing

For efficient search, we can pre-compute the ColBERT representation of each passage and index them.

Below, the `Indexer` take a model checkpoint and writes a (compressed) index to disk. We then prepare a `Searcher` for retrieval from this index.

(With four Titan V GPUs, indexing should take about 13 minutes. The output is fairly long/ugly at the moment!)

In [11]:
nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300   # truncate passages at 300 tokens

checkpoint = 'downloads/colbertv2.0'
index_name = f'{dataset}.{datasplit}.{nbits}bits'

In [12]:
with Run().context(RunConfig(nranks=1, experiment='notebook')):  # nranks specifies the number of GPUs to use.
    config = ColBERTConfig(doc_maxlen=doc_maxlen, nbits=nbits)

    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)



[Feb 23, 01:13:42] #> Note: Output directory /home/adqaicp/documents/ColBERT/docs/experiments/notebook/indexes/lifestyle.dev.2bits already exists


[Feb 23, 01:13:42] #> Will delete 50 files already at /home/adqaicp/documents/ColBERT/docs/experiments/notebook/indexes/lifestyle.dev.2bits in 20 seconds...
#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "index_path": null,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": "bert-base-uncase

0it [00:00, ?it/s]

[Feb 23, 02:32:33] [0] 		 #> Encoding 25000 passages..
[Feb 23, 02:33:23] [0] 		 #> Saving chunk 0: 	 25,000 passages and 3,531,874 embeddings. From #0 onward.


1it [00:57, 57.02s/it]

[Feb 23, 02:33:30] [0] 		 #> Encoding 25000 passages..
[Feb 23, 02:34:20] [0] 		 #> Saving chunk 1: 	 25,000 passages and 3,511,047 embeddings. From #25,000 onward.


...


100%|██████████| 95/95 [00:00<00:00, 241.71it/s]

[Feb 23, 04:02:56] [0] 		 Sorting codes...
[Feb 23, 04:03:18] [0] 		 Getting unique codes...
[Feb 23, 04:03:19] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 23, 04:03:19] #> Building the emb2pid mapping..
[Feb 23, 04:03:24] len(emb2pid) = 331048045



100%|██████████| 262144/262144 [00:36<00:00, 7259.07it/s]

[Feb 23, 04:04:02] #> Saved optimized IVF to /home/adqaicp/documents/ColBERT/docs/experiments/notebook/indexes/lifestyle.dev.2bits/ivf.pid.pt
[Feb 23, 04:04:03] [0] 		 #> Saving the indexing metadata to /home/adqaicp/documents/ColBERT/docs/experiments/notebook/indexes/lifestyle.dev.2bits/metadata.json ..


#> Joined...


In [27]:
indexer = Indexer(checkpoint=checkpoint, config=config)
indexer.index(name=index_name, collection=collection, overwrite=False)

AssertionError: /home/adqaicp/documents/ColBERT/docs/experiments/default/indexes/lifestyle.dev.2bits

In [28]:
indexer.get_index() # You can get the absolute path of the index, if needed.

'/home/adqaicp/documents/ColBERT/docs/experiments/default/indexes/lifestyle.dev.2bits'

## Search

Having built the index and prepared our `searcher`, we can search for individual query strings.

We can use the `queries` set we loaded earlier — or you can supply your own questions. Feel free to get creative! But keep in mind this set of ~300k lifestyle passages can only answer a small, focused set of questions!

In [29]:
# import os
# import sys
# sys.path.insert(0, '../')

# from colbert.infra import Run, RunConfig, ColBERTConfig
# from colbert.data import Queries, Collection
# from colbert import Indexer, Searcher

In [30]:
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

[Feb 23, 04:40:08] #> Loading collection...
0M 1M 2M 
[Feb 23, 04:40:16] #> Loading codec...
[Feb 23, 04:40:16] #> Loading IVF...
[Feb 23, 04:40:16] #> Loading doclens...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [00:00<00:00, 1821.68it/s]

[Feb 23, 04:40:16] #> Loading codes and residuals...



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 95/95 [00:03<00:00, 29.06it/s]


In [23]:
# query = queries[37]   # or supply your own query
# query = "what is applewatch?"
# query = queries[31]
# query = "hello"
# query = "learn SQL"
# query = "prepare a luggage for travel"
query = "Chinese cooking recepit"
# query = queries[3]


print(f"#> {query}")

# Find the top-3 passages for this query 【query数量】
results = searcher.search(query, k=10)
# results = searcher.search(query, k=10)

# Print out the top-k retrieved passages
for passage_id, passage_rank, passage_score in zip(*results):
    print(f"\t [{passage_rank}] \t\t {passage_score:.1f} \t\t {searcher.collection[passage_id]}")

#> Chinese cooking recepit
	 [1] 		 18.6 		 Chinese Cooking. Cookbook.
	 [2] 		 18.6 		 Chinese Cooking. .
	 [3] 		 18.5 		 Dim Sum: Dumplings, Parcels and Other Delectable Chinese Snacks in 25 Authentic Recipes. Dim sum is a traditional style of eating, where bite-sized tidbits are served for shared dining. This book makes authentic Chinese dim sum accessible to the home cook. It opens with a practical introduction to the cuisine, with essential information on ingredients and equipment. More than 25 recipes follow, with a diverse selection of dishes from all over China..
	 [4] 		 18.5 		 The Everything Chinese Cookbook: From Wonton Soup to Sweet and Sour Chicken-300 Succelent Recipes from the Far East. Featuring hundreds of recipes, such as Snow Pea Stir-fry, Hot Chicken Salad, General Tso's Chicken, and Traditional Mu Shu Pork, The Everything Chinese Cookbookmakes preparing authentic Chinese dishes fun and easy! From basic Chinese flavors and dipping sauces, such as Quick and Easy Sw

## Batch Search

In many applications, you have a large batch of queries and you need to maximize the overall throughput. For that, you can use the `searcher.search_all(queries, k)` method, which returns a `Ranking` object that organizes the results across all queries.

(Batching provides many opportunities for higher-throughput search, though we have not implemented most of those optimizations for compressed indexes yet.)

In [16]:
rankings = searcher.search_all(queries, k=5).todict()

100%|████████████████████████████████████| 417/417 [00:01<00:00, 224.98it/s]


In [18]:
rankings[30]  # For query 30, a list of (passage_id, rank, score) for the top-k passages

[(24367, 1, 16.078125),
 (35359, 2, 15.8046875),
 (131545, 3, 15.7421875),
 (3789, 4, 15.7421875),
 (25089, 5, 15.6640625)]